# State Monad

## Outline

* Incentive for the State monad

* Definition of the State monad

* State monad examples

In this lesson, we will learn about the State monad type and how you can use it.

## Incentive for State monad

We talked in the previous two lectures how you can use the Reader and Writer Monad if you have to read from a environment variable or write to it.

If you need to perform both operations you can use the State Monad. Let's first look at an example where we perform read and write operations without the State monad.

We will create the Tic-Tac-Toe game (https://en.wikipedia.org/wiki/Tic-tac-toe) that will use the **System.Random** to randomly pick the X and O choices.

In [ ]:
import System.Random (mkStdGen, Random(randomR), StdGen)

data Player = XPlayer | OPlayer deriving Eq
data Choice = Empty | X | O deriving Eq

data GameState = GameState
  { currentBoard :: [Choice]
  , currentPlayer :: Player
  , generator :: StdGen
  }

main :: IO ()
main = do
    putStrLn "Game results:"
    let gen = mkStdGen 1
        initState = GameState
                      [Empty | boardInd <- [1..9]]
                      XPlayer
                      gen
    playGame initState
    
playGame :: GameState -> IO ()
playGame gs = do
    let freeFields = getFreeFields gs
    if length freeFields /= 0
    then do
        let player = currentPlayer gs
            board = currentBoard gs
            gen = generator gs
            (choiceInd, gen') = randomR (0, length freeFields - 1) gen
            choice = (freeFields !! choiceInd) + 1
            newGameState = GameState
                            (if player == XPlayer
                            then [if ind /= choice then board !! (ind-1) else X | ind <- [1..9]]
                            else [if ind /= choice then board !! (ind-1) else O | ind <- [1..9]])
                            (nextPlayer player)
                            gen'
        playGame newGameState
    else do
        printBoard gs

getFreeFields :: GameState -> [Int]
getFreeFields gs = [ind | ind <- [0..8], board !! ind == Empty]
    where board = currentBoard gs

nextPlayer :: Player -> Player
nextPlayer XPlayer = OPlayer
nextPlayer OPlayer = XPlayer

printBoard :: GameState -> IO ()
printBoard gs = do
    let board = currentBoard gs
    let stateToString st = case st of
                             Empty -> "-"
                             X -> "X"
                             O -> "O"
        printInd ind = stateToString $ board !! ind
    mapM_ putStr [printInd 0,"|", printInd 1,"|", printInd 2, "\n"]
    putStrLn "-----"
    mapM_ putStr [printInd 3,"|", printInd 4,"|", printInd 5, "\n"]
    putStrLn "-----"
    mapM_ putStr [printInd 6,"|", printInd 7,"|", printInd 8, "\n"]

main

In the code above we define the **GameState** variable that holds data for the current state of the game. In the **main** function we create the initial state.

In the **playGame** function we use the **GameState** variable to create a new move and update the variable before we recursivly call this function again or print the game results.

## Definition of the State monad

The definition of the **State** variable is:
```haskell
newtype State s a = State { runState :: s -> (a, s) }
```
We see that the State data constructor holds a function that can be accessed by the name **runState**.

It takes in a state and returns a tuple that contains a variable of type a and another state variable.

## State monad examples

Let's look now at our initial example and how we can re-write it with the use of the State monad.

Just to have an overview we will write out all also the code that is duplicated form the initial example.

In [ ]:
import Control.Monad.State (MonadState(get, put), runState, State)
import System.Random (mkStdGen, Random(randomR), StdGen)

data Player = XPlayer | OPlayer deriving Eq
data Choice = Empty | X | O deriving Eq

data GameState = GameState
  { currentBoard :: [Choice]
  , currentPlayer :: Player
  , generator :: StdGen
  }

main :: IO ()
main = do
  let gen = mkStdGen 1
      initState = GameState
                  [Empty | boardInd <- [1..9]]
                  XPlayer
                  gen
  playGame initState

playGame :: GameState -> IO ()
playGame gs = do
  let (gameFinished, newGS) = runState resolveTurn gs
  if gameFinished 
  then do
    putStrLn "Game results:"
    printBoard newGS
  else playGame newGS

resolveTurn :: State GameState Bool
resolveTurn = do
  choice <- chooseRandomMove
  applyMove choice
  isGameDone

chooseRandomMove :: State GameState Int
chooseRandomMove = do
  gs <- get
  let board = currentBoard gs
      openSpots = [ind | ind <- [0..8], board !! ind == Empty]
      gen = generator gs
  let (i, gen') = randomR (0, length openSpots - 1) gen
  put $ gs { generator = gen' }
  return $ (openSpots !! i) + 1

applyMove :: Int -> State GameState ()
applyMove choice = do
  gs <- get
  let player = currentPlayer gs
      board = currentBoard gs
      newBoard = if player == XPlayer
                 then [if ind /= choice then board !! (ind-1) else X | ind <- [1..9]]
                 else [if ind /= choice then board !! (ind-1) else O | ind <- [1..9]]
  put $ gs { currentPlayer = nextPlayer player, currentBoard = newBoard }

nextPlayer :: Player -> Player
nextPlayer XPlayer = OPlayer
nextPlayer OPlayer = XPlayer

isGameDone :: State GameState Bool
isGameDone = do
  gs <- get
  let board = currentBoard gs
      openSpots = [ind | ind <- [0..8], board !! ind == Empty]
  return $ length openSpots == 0

printBoard :: GameState -> IO ()
printBoard gs = do
    let board = currentBoard gs
    let stateToString st = case st of
                             Empty -> "-"
                             X -> "X"
                             O -> "O"
        printInd ind = stateToString $ board !! ind
    mapM_ putStr [printInd 0,"|", printInd 1,"|", printInd 2, "\n"]
    putStrLn "-----"
    mapM_ putStr [printInd 3,"|", printInd 4,"|", printInd 5, "\n"]
    putStrLn "-----"
    mapM_ putStr [printInd 6,"|", printInd 7,"|", printInd 8, "\n"]

main

Here we use the same **GameState** variable as before that now represents the state of our State monad.

In the **playGame** function we start the **resolveTurn** State monad and provide it the initial state.

This monad calls then three other State monads that update the state accordingly and the final return parameter tells us weather the game is finished.

## Recap

In this lesson we've discussed:

- the motivation for introducing the State monad type 

- the definition of the State monad type and a simple example

- examples that use the State monad type to solve a problem